In [1]:
from utils.logger import setup_logger
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
import config
import time
import bs4
import re

# Set up the logger
logger = setup_logger()

In [3]:
# Set up Chrome options (optional: you can add more options as needed)
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode (no GUI)
chrome_service = Service("/home/xbankov/COP_analysis/chromedriver-linux64/chromedriver")

# Initialize the Chrome brexecutable_pathowser
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)
driver.implicitly_wait(20)

# Replace with the URL you want to open
url_to_open = config.TEST_URL

# Open the URL
driver.get(url_to_open)

items_per_page_button = driver.find_element(By.ID, "edit-items-per-page--4")
select = Select(items_per_page_button)
last_index = len(select.options) - 1
select.select_by_index(last_index)

loading_div = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "div.ajax-progress"))
)
print("Loading started")
# Wait until the div element disappears
WebDriverWait(driver, 30).until(EC.staleness_of(loading_div))
print("Loading done")

total_documents = int(
    driver.find_elements(By.CSS_SELECTOR, "span.totalresults")[-1].text
)
shown_documents = int(driver.find_elements(By.CSS_SELECTOR, "span.endresults")[-1].text)

while shown_documents < total_documents:
    print(f"{shown_documents}/{total_documents}")

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)

    load_more_button = driver.find_element(
        By.CSS_SELECTOR,
        'div.block-views-block-documents-block-1 a.button[title="Load more items"]',
    )
    load_more_button.click()

    loading_div = WebDriverWait(driver, 30).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "div.ajax-progress"))
    )
    print("Loading started")
    # Wait until the div element disappears
    WebDriverWait(driver, 30).until(EC.staleness_of(loading_div))
    print("Loading done")

    shown_documents = int(
        driver.find_element(
            By.CSS_SELECTOR, "div.block-views-block-documents-block-1 span.endresults"
        ).text
    )
print("Done")
html_content = driver.page_source

Loading started
Loading done
50/74
Loading started
Loading done
Done


In [456]:
html_content = driver.page_source
soup = bs4.BeautifulSoup(html_content)
documents = soup.find_all("tr")[1:]

In [457]:
len(documents)

310

In [458]:
def get_eng_url_from_td(td):
    select_element = td.select_one(".select-wrapper select")
    english_option = select_element.find(
        "option", string=re.compile("english", re.IGNORECASE)
    )
    return english_option["value"]

In [461]:
document = documents[-1]
cols = document.find_all("td")
title = cols[1].getText().strip().replace(" ", "_")
download_url = get_eng_url_from_td(cols[4])

In [462]:
print(title) 
print(download_url)

Draft_report_of_the_Subsidiary_Body_for_Scientific_and_Technological_Advice_on_its_fifty-second_to_fifty-fifth_session
https://unfccc.int/sites/default/files/resource/sbsta2021_L01_E.pdf


In [463]:
import requests

In [130]:
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.3',
}
response = requests.get(download_url, headers=headers)

if response.status_code == 200:
    # Save the PDF content to a local file
    with open(f"{download_url.split('/')[-1]}", 'wb') as pdf_file:
        pdf_file.write(response.content)
    print(f"Downloaded: {download_url}")
else:
    print(f"Failed to download: {download_url}")

Downloaded: https://unfccc.int/sites/default/files/resource/cp2023_L10r01E.pdf
